# 多输入和多输出通道
:label:`sec_channels`

虽然我们在:numref:`subsec_why-conv-channels`中描述了构成每个图像的多个通道（例如，彩色图像有标准的RGB通道来表示红色、绿色和蓝色的数量）以及多通道的卷积层，但到目前为止，我们通过只使用单个输入和单个输出通道简化了所有的数值示例。这使我们可以将输入、卷积核和输出都视为二维张量。

当我们加入通道时，我们的输入和隐藏表示都变成了三维张量。例如，每个RGB输入图像的形状为$3\times h\times w$。我们称这个大小为3的轴为*通道*维度。通道的概念与CNN本身一样古老：例如LeNet-5 :cite:`LeCun.Jackel.Bottou.ea.1995`就使用了它们。在本节中，我们将更深入地探讨具有多个输入和多个输出通道的卷积核。

In [1]:
import torch
from d2l import torch as d2l

## 多输入通道

当输入数据包含多个通道时，
我们需要构建一个卷积核，
其输入通道数与输入数据相同，
以便能够与输入数据进行互相关运算。
假设输入数据的通道数为 $c_\textrm{i}$，
那么卷积核的输入通道数也需要是 $c_\textrm{i}$。如果我们的卷积核窗口形状是 $k_\textrm{h}\times k_\textrm{w}$，
那么，当 $c_\textrm{i}=1$ 时，我们可以认为我们的卷积核
只是一个形状为 $k_\textrm{h}\times k_\textrm{w}$ 的二维张量。

然而，当 $c_\textrm{i}>1$ 时，我们需要一个卷积核，
它包含每个输入通道的一个形状为 $k_\textrm{h}\times k_\textrm{w}$ 的张量。将这些 $c_\textrm{i}$ 个张量连接在一起，
就得到一个形状为 $c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$ 的卷积核。
由于输入和卷积核各有 $c_\textrm{i}$ 个通道，
我们可以在每个通道上对输入的二维张量
和卷积核的二维张量执行互相关操作，
然后将 $c_\textrm{i}$ 个结果相加
（在通道上求和）
以得到一个二维张量。
这就是多通道输入与
多输入通道卷积核之间的二维互相关的结果。

:numref:`fig_conv_multi_in` 提供了一个具有两个输入通道的二维互相关的例子。
阴影部分是第一个输出元素
以及用于计算输出的输入和卷积核张量的元素：
$(1\times1+2\times2+4\times3+5\times4)+(0\times0+1\times1+3\times2+4\times3)=56$。

![Cross-correlation computation with two input channels.](../img/conv-multi-in.svg)
:label:`fig_conv_multi_in`


为了确保我们真正理解这里发生的事情，
我们可以自己实现具有多个输入通道的互相关操作。
请注意，我们所做的只是对每个通道执行互相关操作，
然后将结果相加。

In [2]:
def corr2d_multi_in(X, K):
    # Iterate through the 0th dimension (channel) of K first, then add them up
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

我们可以根据:numref:`fig_conv_multi_in`中的值来构建输入张量`X`和核张量`K`，以（**验证互相关操作的输出**）。

In [3]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

## 多个输出通道
:label:`subsec_multi-output-channels`

无论输入通道的数量是多少，
到目前为止，我们最终总是得到一个输出通道。
然而，正如我们在 :numref:`subsec_why-conv-channels` 中讨论的那样，
在每一层拥有多个通道是非常重要的。
在最流行的神经网络架构中，
实际上随着神经网络的深入，我们增加了通道维度，
通常通过降低空间分辨率来换取更大的*通道深度*。
直观地讲，你可以认为每个通道
对不同的特征集作出响应。
实际情况比这要复杂一些。一种简单的解释会暗示
表示是按像素或按通道独立学习的。
实际上，通道被优化为共同发挥作用。
这意味着，与其将单个通道映射到边缘检测器，不如说
在通道空间中的某个方向对应于检测边缘。

设 $c_\textrm{i}$ 和 $c_\textrm{o}$ 分别为
输入和输出通道的数量，
$k_\textrm{h}$ 和 $k_\textrm{w}$ 为卷积核的高度和宽度。
为了获得具有多个通道的输出，
我们可以为*每一个*输出通道创建一个形状为 $c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$ 的卷积核张量。
我们将它们在输出通道维度上连接起来，
使得卷积核的形状为 $c_\textrm{o}\times c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$。
在互相关操作中，
每个输出通道的结果
是由对应于该输出通道的卷积核计算得出的，
并且从输入张量的所有通道中获取输入。

我们实现了一个互相关函数
以[**计算多个通道的输出**]如下所示。

In [4]:
def corr2d_multi_in_out(X, K):
    # Iterate through the 0th dimension of K, and each time, perform
    # cross-correlation operations with input X. All of the results are
    # stacked together
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

我们通过将`K`、`K+1`和`K+2`的核张量连接起来，构建了一个具有三个输出通道的简单卷积核。

In [5]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

在下面的操作中，我们对输入张量`X`和核张量`K`执行互相关操作。现在输出包含三个通道。第一个通道的结果与之前的输入张量`X`和多输入通道、单输出通道的核的结果一致。

In [6]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

## $1\times 1$ 卷积层
:label:`subsec_1x1`

乍一看，[**$1 \times 1$ 卷积**]，即 $k_\textrm{h} = k_\textrm{w} = 1$，似乎没有多大意义。毕竟，卷积关联相邻的像素。而 $1 \times 1$ 卷积显然做不到这一点。尽管如此，它们仍然是有时会包含在复杂深度网络设计中的流行操作 :cite:`Lin.Chen.Yan.2013,Szegedy.Ioffe.Vanhoucke.ea.2017`。让我们详细看看它实际上做了什么。

由于使用了最小窗口，$1\times 1$ 卷积失去了较大的卷积层识别高度和宽度维度上相邻元素相互作用模式的能力。$1\times 1$ 卷积的唯一计算发生在通道维度上。

:numref:`fig_conv_1x1` 显示了使用具有 3 个输入通道和 2 个输出通道的 $1\times 1$ 卷积核进行互相关计算的情况。请注意，输入和输出的高度和宽度相同。输出中的每个元素都来源于输入图像中*同一位置*的元素的线性组合。你可以将 $1\times 1$ 卷积层视为在每个像素位置应用的一个全连接层，将 $c_\textrm{i}$ 个对应的输入值转换为 $c_\textrm{o}$ 个输出值。因为这仍然是一个卷积层，所以权重在像素位置之间是共享的。因此，$1\times 1$ 卷积层需要 $c_\textrm{o}\times c_\textrm{i}$ 个权重（加上偏置）。还要注意的是，卷积层通常后面跟着非线性激活函数。这确保了 $1 \times 1$ 卷积不能简单地折叠到其他卷积中。

![The cross-correlation computation uses the $1\times 1$ convolution kernel with three input channels and two output channels. The input and output have the same height and width.](../img/conv-1x1.svg)
:label:`fig_conv_1x1`

让我们检查一下这在实践中是否有效：我们使用全连接层实现一个 $1 \times 1$ 卷积。唯一需要注意的是，在矩阵乘法之前和之后我们需要对数据形状做一些调整。

In [7]:
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # Matrix multiplication in the fully connected layer
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

在执行$1\times 1$卷积时，上述函数等同于之前实现的交叉相关函数`corr2d_multi_in_out`。让我们用一些示例数据来检查这一点。

In [8]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))
Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

## 讨论

通道让我们能够结合两者的优点：允许显著非线性的多层感知机（MLP）和允许对特征进行*局部*分析的卷积。特别是，通道使CNN能够同时处理多种特征，如边缘和形状检测器。它们还提供了由平移不变性和局部性带来的大幅参数减少与计算机视觉中需要表达力强且多样化的模型之间的实际折衷。

需要注意的是，这种灵活性是有代价的。给定一个大小为$(h \times w)$的图像，计算$k \times k$卷积的成本是$\mathcal{O}(h \cdot w \cdot k^2)$。对于$c_\textrm{i}$个输入通道和$c_\textrm{o}$个输出通道，这增加到$\mathcal{O}(h \cdot w \cdot k^2 \cdot c_\textrm{i} \cdot c_\textrm{o})$。对于一个$256 \times 256$像素的图像，使用$5 \times 5$内核以及分别有$128$个输入和输出通道的情况下，这相当于超过530亿次操作（我们单独计算乘法和加法）。稍后我们将遇到有效的策略来降低成本，例如要求通道间的操作是块对角的，从而导致如ResNeXt :cite:`Xie.Girshick.Dollar.ea.2017`这样的架构。

## 练习

1. 假设我们有两个大小分别为$k_1$和$k_2$的卷积核
   （中间没有非线性）。
    1. 证明该操作的结果可以用单个卷积表示。
    1. 等效单个卷积的维度是多少？
    1. 反之亦然吗？即，你能否总是将一个卷积分解成两个较小的卷积？
1. 假设输入形状为$c_\textrm{i}\times h\times w$，卷积核形状为
   $c_\textrm{o}\times c_\textrm{i}\times k_\textrm{h}\times k_\textrm{w}$，填充为$(p_\textrm{h}, p_\textrm{w})$，步幅为$(s_\textrm{h}, s_\textrm{w})$。
    1. 前向传播的计算成本（乘法和加法）是多少？
    1. 内存占用是多少？
    1. 反向计算的内存占用是多少？
    1. 反向传播的计算成本是多少？
1. 如果我们将输入通道数$c_\textrm{i}$和输出通道数$c_\textrm{o}$都翻倍，计算量会增加多少倍？如果我们将填充翻倍会发生什么？
1. 在本节最后一个示例中，变量`Y1`和`Y2`是否完全相同？为什么？
1. 即使当卷积窗口不是$1 \times 1$时，也要将卷积表示为矩阵乘法。
1. 你的任务是实现具有$k \times k$内核的快速卷积。其中一个算法候选方案
   是水平扫描源数据，读取宽度为$k$的条带并逐值计算宽度为$1$的输出条带。
   另一种方法是读取宽度为$k + \Delta$的条带并计算宽度为$\Delta$的输出条带。为什么后者更优？选择$\Delta$是否有上限？
1. 假设我们有一个$c \times c$矩阵。
    1. 如果矩阵被分成$b$块，与块对角矩阵相乘快多少？
    1. 有$b$块有什么缺点？你至少可以部分地如何解决这个问题？

[讨论](https://discuss.d2l.ai/t/70)